<a href="https://colab.research.google.com/github/dtabuena/CrispyMice/blob/main/New_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)
'''Get Repositories'''


'Get Repositories'

In [2]:

try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib

to_import = [
          'ABF_Quality_Control.ipynb',
          'Basic_Ephys.ipynb',
          'Firing_Rate_Gain.ipynb',
          'Simple_ABF_tools.ipynb',
          'IV_analyzer.ipynb',
          'Vm_analyzer.ipynb',
          'membrane_analyzer.ipynb',
          'analyze_rheobase.ipynb',
          'fun_math.ipynb',
          'importing_abfs_from_dropbox.ipynb',
          'input_resistance_analyzer.ipynb',
          'latencey_analyzer.ipynb',
          'QC_recoding_dataframe.ipynb'
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


try: shutil.rmtree('/content/CrispyMice')
except: None
!git clone https://github.com/dtabuena/CrispyMice
to_import = ['CrispyPipelineFuncs.ipynb']
for i in to_import:
    f = '/content/CrispyMice/' + i
    %run $f

Cloning into 'EphysLib'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 247 (delta 102), reused 29 (delta 29), pack-reused 94
Receiving objects: 100% (247/247), 204.58 KiB | 22.73 MiB/s, done.
Resolving deltas: 100% (146/146), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 
Cloning into 'CrispyMice'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 147 (delta 35), reused 25 (delta 25), pack-reused 107
Receiving objects: 100% (147/147), 3.48 MiB | 27.87 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [3]:
dataset = {'data_name': 'RNF182',
           'data_source': "https://www.dropbox.com/sh/n9t8p257wnzlijk/AAC9Z36JodisyZjnM3mkJC3Xa?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree']

In [4]:
'''Unpack'''
data_name = dataset['data_name']
data_source = dataset['data_source']
file_naming_scheme = dataset['file_naming_scheme']


'''Set Internal Analysis Params'''
spike_args =  {'spike_thresh':20, 'high_dv_thresh': 25,'low_dv_thresh': -5,'window_ms': 2}

func_dict = {}
arg_dict = {}

func_dict['VC - 3min GapFree']= rmp_analyzer
arg_dict['VC - 3min GapFree'] = [False]

func_dict['IC - Rheobase']= rheobase_analyzer
arg_dict['IC - Rheobase'] = [spike_args, False, False, False]  # [spike_args, to_plot, verbose, force_singlespike]

func_dict['IC - Gain - D20pA']= gain_analyzer
arg_dict['IC - Gain - D20pA']= [spike_args, 0.8, 0]  # [spike_args, to_plot, verbose, force_singlespike]
func_dict['IC - Gain - D50pA']= func_dict['IC - Gain - D20pA'] 
arg_dict['IC - Gain - D50pA']= arg_dict['IC - Gain - D20pA']

func_dict['VC - MemTest-10ms-160ms']= membrane_analyzer
arg_dict['VC - MemTest-10ms-160ms']= [False, False, ['Ra', 'Rm', 'Cm', 'tau',	'Cmq',	'Cmf',	'Cmqf', 'Cm_pc']]  # [to_plot, verbose]

func_dict['IC - Latentcy 800pA-1s']= latencey_analyzer 
arg_dict['IC - Latentcy 800pA-1s']= [spike_args, False]  # [spike_args, to_plot]

func_dict['IC - Latentcy 800pA-1s']= latencey_analyzer 
arg_dict['IC - Latentcy 800pA-1s']= [spike_args, False]  # [spike_args, to_plot]

func_dict['IC - R input']= input_resistance_analyzer 
arg_dict['IC - R input']= [[-30, 10] ,False]  # [dVm_limits, to_plot]

func_dict['VC - Multi IV - 150ms'] = IV_analyzer
arg_dict['VC - Multi IV - 150ms']= [{'IV_Early':(16.5, 30),'IV_Steady_State':(100,120)} ,False]  # [measure_windows, to_plot]


''' Gather and Catalog Source Data'''
file_loc = get_drobox_folder(data_source, 'my_ephys_data')
clear_output(wait=False)   
abf_recordings_df, protocol_set = catalogue_recs(file_loc,file_naming_scheme)
abf_recordings_df, _ = purge_wrong_clamp(abf_recordings_df,VC_prot,IC_prot)
cell_prot_lut(abf_recordings_df,protocol_set,csv_name=data_name+'_LUT')


'''Analyze Dataset'''
abf_recordings_df, problem_recs = analysis_iterator(abf_recordings_df,func_dict,arg_dict)
print(problem_recs) 

cell_df = cell_sorting(abf_recordings_df)

'''Consolidate to Cells'''
list_types = ['Recording_name','protocol','abf_timestamp', 'channelList']
any_types = ['Rec_date',	'Virus',	'GenoType',	'Sex',	'Age',	'Slice_Num',  'Cell_num', 'Cell_Type']
cell_df_con = cell_consolidation(cell_df,list_types,any_types)
cols_to_simplify = ['IV_Early', 'IV_Steady_State']
cell_df_nd = simplify_dicts(cell_df_con,cols_to_simplify)     

'''Make Excell Friendly'''
keys_and_data_cols={'Stim_Levels_(pA)': ['Stim_Levels_(pA)', 'Spike_Counts' ],
                'IV_Early_(V_stim)': ['IV_Early_(V_stim)', 'IV_Early_(I_peak)', 'IV_Steady_State_(I_mean)']}
cell_df_csv = csv_frinedly(cell_df_nd,keys_and_data_cols)

''' Convert to Current Density'''
size_col = 'Cmq_160.0'
current_col_list = ['IV_Early_(I_peak)_', 'IV_Steady_State_(I_mean)_']
cell_df_csv = current_density_correction(cell_df_csv, size_col, current_col_list)


'''Abridge DataFrame'''
abrg_exclusions = ['Recording_name', 
                'protocol', 'abf_timestamp', 'channelList',  'Ra_10.0', 'Rm_10.0', 'tau_10.0', 'Cmq_10.0', 'Cmf_10.0',
                'Cmqf_10.0',  'Cmf_160.0', 'Cmqf_160.0', 'Cm_pc_160.0',
                'Gain_R2', 'Stim_Levels_(pA)', 'Spike_Counts', 'Firing_Duration_%', 'Gain_Vh',  'Vhold_spike',
                    'Rin_Rsqr',  'Ramp_AP_thresh', 'Ramp_Vh', 'Ramp_Rheobase', 
                'ap_thresh_us', 'v_half',
                'IV_Early_(range)', 'IV_Early_(I_peak)', 'IV_Early_(I_mean)', 'IV_Early_(V_stim)', 'IV_Steady_State_(range)',
                'IV_Steady_State_(I_peak)', 'IV_Steady_State_(I_mean)', 'IV_Steady_State_(V_stim)', ]

abrg_keep = [c for c in cell_df_csv.columns if c not in abrg_exclusions]
cell_df_csv_abrg = cell_df_csv[abrg_keep]

'''Stratify Cells By Type'''
strat_col = 'Cell_Type'
xl_file_name= data_name+'.xlsx'
new_dfs = stratify_cells(cell_df_csv_abrg,strat_col,xl_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  4%|▍         | 10/251 [00:09<04:31,  1.13s/it]

ap_stats_failed:  ABF (v2.9) with 2 channels (mV, pA), sampled at 10.0 kHz, containing 3 sweeps, having no tags, with a total length of 0.57 minutes, recorded with protocol "IC - Rheobase".


 24%|██▍       | 60/251 [00:42<01:44,  1.83it/s]

...

ap_stats_failed:  ABF (v2.9) with 2 channels (mV, pA), sampled at 10.0 kHz, containing 7 sweeps, having no tags, with a total length of 1.23 minutes, recorded with protocol "IC - Rheobase".


100%|██████████| 251/251 [03:10<00:00,  1.31it/s]


['my_ephys_dat/2022_08_12_RNF182/2022x08x12_RNF182_E4KI_F_P251_s002_c006_CA3xPOS_0006.abf']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
cell_df_csv_abrg.columns[:6]

Index(['Rec_date', 'Virus', 'GenoType', 'Sex', 'Age', 'Slice_Num'], dtype='object')

In [6]:
strat_col = ['Cell_Type', 'Rec_date']

strat_df_dict = stratify_rec(cell_df_csv_abrg,strat_col)




In [20]:
def flatten_dict(my_dict,flat_dict = {} ):
    for k in my_dict.keys():
        if isinstance(my_dict[k], dict):
            sub_dict, sub_keys = flatten_dict(my_dict[k],flat_dict)
            for sk in sub_keys:
                flat_dict[k+'_'+sk] = sub_dict[sk]
        else:
            flat_dict = my_dict
    return flat_dict, list(flat_dict.keys()) 

ad = {'a1': 1, 'a2': 2}
bd = {'b1': 3, 'b2': 4}
a ={ 'a': ad, 'b': bd }

print(a)
fd, fdk = flatten_dict(a)
print(fd)

{'a': {'a1': 1, 'a2': 2}, 'b': {'b1': 3, 'b2': 4}}
{'a_a1': 1, 'a_a2': 2, 'b_b1': 3, 'b_b2': 4}


{'a': {'a1': 1, 'a2': 2}, 'b': {'b1': 3, 'b2': 4}}
flat {}
True
flat {}
False
False
sub_dict {}
True
flat {}
False
False
sub_dict {}
{}
